# Importar bibliotecas

In [ ]:
import pandas as pd
import numpy as np

np.set_printoptions(threshold=None, precision=2)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('precision', 2)
pd.set_option('float_format', '{:.2f}'.format)

# Carregando arquivos lote e descartes

## Lote

In [ ]:
df_lote = pd.read_csv('Lote.csv')
print(f'Dimensões do Lote:\n{df_lote.shape[0]} linhas')
print(f'Campos do Lote:\n{list(df_lote.columns)}\n')
df_lote.info()

## Descartes

In [ ]:
df_descartes = pd.read_csv('Descartes.csv', encoding='latin')
print(f'Dimensões do Lote:\n{df_descartes.shape[0]} linhas\n')
print(f'Campos do Lote:\n{list(df_descartes.columns)}\n')
df_descartes.info()

### Manipulação dos dados originais

#### Criar colunas com o tipo de NI para os beneficiários e os interessados dos processos

In [ ]:
#Tipos de NIs
df_lote['Tipo_NI_Ben'] = df_lote['NI Beneficiário'].apply(lambda x: 'PJ' if len(x) == 18 else 'PF')
df_lote['Tipo_NI_Int'] = df_lote['NI Interessado'].apply(lambda x: 'PJ' if len(x) == 18 else 'PF')

#NIs iguais
condicao = df_lote['NI Beneficiário'].values==df_lote['NI Interessado'].values
df_lote['NIs_iguais'] = np.where(condicao, 'S', 'N')

#Ano do protocolo
df_lote['Processo'] = df_lote['Processo'].astype(str)
df_lote['Ano'] = df_lote['Processo'].str.slice(14,-3).astype(int)

#Tempo decorrido
condicao = [
    (df_lote['Ano'].values < 100) & (df_lote['Ano'].values > 0),
    (df_lote['Ano'].values == 0),
    (df_lote['Ano'].values > 100)
    ]

escolhas = [
    2020 - (df_lote['Ano'].values + 1900),
    2020 - (df_lote['Ano'].values + 2000),
    2020 - df_lote['Ano'].values
           ]

df_lote['Tempo_decorrido'] = np.select(condicao, escolhas,'a')

# Filtro para restituições cujo beneficiário seja PF

In [ ]:
df_lote = df_lote.loc[df_lote['Tipo_NI_Ben']=='PF']

# Dicionário para renomear processos e NIs

## Remoção das pontuações dos campos "Processo" e "Número do Processo"

In [ ]:
df_lote['Processo'] = df_lote['Processo'].str.replace(r'[^\w]', '', regex=True).astype(str)
df_descartes['Número do Processo'] = df_descartes['Número do Processo'].str.replace(r'[^\w]', '', regex=True).astype(str)

## Criação de dicionário com todos os números de processos, NI e novos índices

In [ ]:
#Criação dos dicionários
lista_de_processos = np.append(df_lote['Processo'].unique(), df_descartes['Número do Processo'].unique())
lista_de_processos = list(set(lista_de_processos))
lista_indice = list(range(len(lista_de_processos)))

dicionario_Processo = {proc1:proc2 for proc1, proc2 in zip(lista_de_processos, lista_indice)}

lista_de_cpf = df_lote['NI Beneficiário'].unique()

dicionario_NI = {ni:ni2 for ni, ni2 in zip(list(set(lista_de_cpf)),range(len(list(set(lista_de_cpf)))))}

# Alterações para tornar sigilosos

##  df_lote

In [ ]:
#Alteração no dataframe df_lote
df_lote['NI Beneficiário'] = df_lote['NI Beneficiário'].replace(dicionario_NI)
df_lote['Processo'] = df_lote['Processo'].replace(dicionario_Processo)

## df_descartes

In [ ]:
df_descartes['Número do Processo'] = df_descartes['Número do Processo'].replace(dicionario_Processo)

## Remoção das colunas "Nome/Razão Social", "CPF do Usuário do Impedimento" e "CPF/CNPJ"

In [ ]:
df_descartes = df_descartes.drop(columns=['Nome/Razão Social', 'CPF do Usuário do Impedimento','CPF/CNPJ'])

 ## Remoção da coluna "NI Interessado"

In [ ]:
df_lote = df_lote.drop(columns=['NI Interessado'])

# Exportar df_lote e df_descartes

In [ ]:
df_lote.to_csv('Lote monografia sigilo.csv', index=False)
df_descartes.to_csv('Descartes monografia sigilo.csv', index=False)

## Leitura/limpeza e exportação do arquivo de informação dos processos

In [ ]:
#Carregar arquivo no formato excel.
df_processos = pd.read_excel('Processos.xlsx')

#Remover atributo que requer sigilo
df_processos = df_processos.drop(columns='CNPJ/CPF')

#Remover pontuações do atributo e substituir valores conforme dicionário criado
df_processos['Nº Processo'] = df_processos['Nº Processo'].str.replace(r'[^\w]', '', regex=True)
df_processos['Nº Processo'] = df_processos['Nº Processo'].replace(dicionario_Processo)

#Exportar dados para arquivo ".csv"
df_processos.to_csv('Processos_PF_monografia.csv', index=False)

## Leitura/limpeza e exportação do arquivo de cadastros

In [ ]:
#Carregar arquivo no formato excel.
df_cadastros = pd.read_excel('Cadastros.xls')

#Substituir valores conforme dicionário criado
df_cadastros['CPF'] = df_cadastros['CPF'].replace(dicionario_NI)

#Remover atributo que requer sigilo
colunas_remover = ['Nome', 'Logradouro', 'Número', 'Complemento', 'Bairro/Distrito', 'Caixa Postal', 'CEP',
                   'Correio Eletrônico', 'Telefone - Número', 'Fax - Número', 'Nome da Mãe', 'NIT', 
                   'Título de Eleitor', 'CPF do Representante']
df_cadastros = df_cadastros.drop(columns=colunas_remover)

#Exportar dados para arquivo ".csv"
df_cadastros.to_csv('Cadastro_PF_monografia.csv', index=False)